#### 从Assets/ExpLog中提取json文件进行处理. 结果暂存为.csv以方便使用excel.

**注意json中的firsttouch的目标不可靠，坐标应该对应inputsequence中删去-+的区域**

In [29]:
# 导入包，设置路径
import numpy as np
from openpyxl import Workbook
import os
import json
import csv
from EditDistance import EditDistance

LogDir = "D:/Unity/Project/Keyboard/Assets/ExpLog/"

In [30]:
# 获取json文件列表，并以名称为索引
tmpfList = os.listdir(LogDir)
fDist = {}   # "name" : [0's filename, 1's filename, 2's filename, 3's filename..]
nameList = []
typeList = ["NSPad", "NSPad+Crossover", "FanPad", "FanPad+Crossover"]
typenum = 4

for fname in tmpfList:
    if fname.endswith("meta"):
        continue
    n, t, time = fname[:-5].split('_')
    t = int(t)
    try:
        nameList.index(n)
    except ValueError:
        nameList.append(n)
    if n not in fDist.keys():
        fDist[n] = [None, None, None, None]
    if fDist[n][t] is not None:
        _, _, another_time = fname[:-5].split('_')
        if another_time < time:
            # 使用更晚的.
            fDist[n][t] = fname
    else:
        fDist[n][t] = fname

In [31]:
print(nameList)
print(fDist)

['cx', 'cyl', 'dyh', 'fjy', 'lhz', 'ljh', 'lyc', 'qym', 'wzt', 'zby', 'zzc']
{'cx': ['cx_0_20230906145018.json', 'cx_1_20230906145940.json', 'cx_2_20230906152153.json', 'cx_3_20230906153834.json'], 'cyl': ['cyl_0_20230905173205.json', 'cyl_1_20230905173826.json', 'cyl_2_20230906175823.json', 'cyl_3_20230906180500.json'], 'dyh': ['dyh_0_20230904205330.json', 'dyh_1_20230904210151.json', 'dyh_2_20230904211745.json', 'dyh_3_20230904212638.json'], 'fjy': ['fjy_0_20230906161701.json', 'fjy_1_20230906162310.json', 'fjy_2_20230903205052.json', 'fjy_3_20230906164216.json'], 'lhz': ['lhz_0_20230906210930.json', 'lhz_1_20230906211641.json', 'lhz_2_20230906213738.json', 'lhz_3_20230903215357.json'], 'ljh': ['ljh_0_20230905192925.json', 'ljh_1_20230905194937.json', 'ljh_2_20230905200845.json', 'ljh_3_20230905203547.json'], 'lyc': ['lyc_0_20230904142102.json', 'lyc_1_20230904142651.json', 'lyc_2_20230904152622.json', 'lyc_3_20230904153338.json'], 'qym': ['qym_0_20230905165251.json', 'qym_1_20230905

In [32]:
# 准备一些需要使用的函数.
def TotalError(phrases:list[str], inputSequence: list[str]):
    # 计算输入过程中的总错误数..先还是用下标按顺序来吧..
    allPhrase = "\n".join(phrases).lower()
    cur = ""
    ind = 0
    err = 0
    for next in inputSequence:
        next = next.lower()
        if next == "back":
            cur = cur[:-1]
            ind = 0 if ind == 0 else ind - 1
            next = ""
        elif next == "sym" or next == "shift":
            # 直接忽略
            next = ""
        elif next[0] == '-':
            delete, add = next.split(", ")
            cur = cur[:-(len(delete)-1)] + add[1:]
            ind = ind - len(delete) + 1
            next = add[1:]
        else:
            cur = cur + next
        # 接下来是比较.
        for j in range(len(next)):
            if allPhrase[i] != next[j]:
                if allPhrase[i] == "\n" and next[j] == ' ':
                    i -= 1
                else:
                    err += 1
            i += 1
    return err

# 用back总次数算.
def BackTimes(inputSequence : list[str]):
    err = 0
    for i in inputSequence:
        if i == "Back":
            err += 1
    return err

In [33]:
# 保存数据到xlsx中.
head = ['name', 'time', 'length', 'WPM', 'TER', 'NCER']

wb = Workbook()

for i in range(typenum):
    rows = []
    ws = wb.create_sheet(typeList[i], i)
    ws.append(head)
    for user in nameList:
        # print(user, i)
        file = fDist[user][i]
        if file is None:
            continue
        else:
            file = os.path.join(LogDir, file)
            with open(file, "r") as f:
                jsonFile = json.load(f)
            time = jsonFile["seconds"]
            length = jsonFile["phraseLength"]
            WPM = jsonFile["WPM"]
            # 算错误率
            phrases = jsonFile["phrases"]
            phrases = ' '.join(phrases)
            result = jsonFile["result"].split("\n")
            for j in range(len(result)):
                result[j] = result[j].strip()
            result = ' '.join(result)
            # print(phrases)
            # print(result)
            ncerr = EditDistance(phrases, result)
            # for j in range(len(phrases)):
            #     result[j] = result[j].strip()
            #     ncerr += EditDistance(phrases[j], result[j])
            totalerr = ncerr + BackTimes(jsonFile["inputSequence"])
            rows.append([user, time, length, WPM, totalerr/length, ncerr/length])
            ws.append([user, time, length, WPM, totalerr/length, ncerr/length])
    # 保存到csv中
    # with open(os.path.join("./", typeList[i] + "_speed_err.csv"), "w", encoding="utf-8-sig", newline="") as f:
    #     csv_writer = csv.writer(f)
    #     csv_writer.writerow(head)
    #     csv_writer.writerows(rows)
wb.save(os.path.join("./", "Speed_Err.xlsx"))